In [1]:
from transformers import pipeline

nlp=pipeline("sentiment-analysis")
result=nlp("I hate you")[0]
print(f"label: {result['label']}, with score: {round(result['score'],4)}")
result=nlp("I love you")[0]
print(f"label: {result['label']}, with score: {round(result['score'],4)}")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


label: NEGATIVE, with score: 0.9991
label: POSITIVE, with score: 0.9999


In [2]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification,pipeline

model_name="nlptown/bert-base-multilingual-uncased-sentiment"
model=AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer=AutoTokenizer.from_pretrained(model_name)
classifier=pipeline('sentiment-analysis',model=model,tokenizer=tokenizer)

classifier("I love you")

[{'label': '5 stars', 'score': 0.8546808362007141}]

In [3]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
import torch
tokenizer=AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model=AutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")
classes=["not paraphrase","is paraphrase"]
sequence_0="The company HuggingFace is based in New York City"
sequence_1 = "Apples are especially bad for your health"
sequence_2 = "HuggingFace's headquarters are situated in Manhattan"
paraphrase=tokenizer(sequence_0,sequence_2,return_tensors="pt")
not_paraphrase=tokenizer(sequence_0,sequence_1,return_tensors="pt")
paraphrase_classification_logits=model(**paraphrase).logits
not_paraphrase_classification_logits=model(**not_paraphrase).logits
paraphrase_results=torch.softmax(paraphrase_classification_logits,dim=1).tolist()[0]
not_paraphrase_results=torch.softmax(not_paraphrase_classification_logits,dim=1).tolist()[0]

for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(paraphrase_results[i]*100))}%")
for i in range(len(classes)):
	print(f"{classes[i]}: {int(round(not_paraphrase_results[i]*100))}%")

not paraphrase: 10%
is paraphrase: 90%
not paraphrase: 94%
is paraphrase: 6%


In [4]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification

model_name="distilbert-base-uncased-finetuned-sst-2-english"
model=AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer=AutoTokenizer.from_pretrained(model_name)
inputs=tokenizer("We are very happy to show you the 🤗 Transformers library.")
inputs

{'input_ids': [101, 2057, 2024, 2200, 3407, 2000, 2265, 2017, 1996, 100, 19081, 3075, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [5]:
batch=tokenizer(
    ["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."],
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt"
)
for key,value in batch.items():
    print(f"{key}: {value.numpy().tolist()}")

input_ids: [[101, 2057, 2024, 2200, 3407, 2000, 2265, 2017, 1996, 100, 19081, 3075, 1012, 102], [101, 2057, 3246, 2017, 2123, 1005, 1056, 5223, 2009, 1012, 102, 0, 0, 0]]
attention_mask: [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]]


In [6]:
outputs=model(**batch)
print(outputs)

SequenceClassifierOutput(loss=None, logits=tensor([[-4.0833,  4.3364],
        [ 0.0818, -0.0418]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [7]:
import torch.nn.functional as F
predictions=F.softmax(outputs[0],dim=-1)
predictions

tensor([[2.2043e-04, 9.9978e-01],
        [5.3086e-01, 4.6914e-01]], grad_fn=<SoftmaxBackward0>)

In [8]:
tokenizer.save_pretrained("E:/Study/Pythonlearning/Transformerslearning/hello_state1")
model.save_pretrained("E:/Study/Pythonlearning/Transformerslearning/hello_state1")

In [9]:
from transformers import AutoTokenizer,AutoModel
tokenizer=AutoTokenizer.from_pretrained("E:/Study/Pythonlearning/Transformerslearning/hello_state1")
model=AutoModel.from_pretrained("E:/Study/Pythonlearning/Transformerslearning/hello_state1")

Some weights of the model checkpoint at E:/Study/Pythonlearning/Transformerslearning/hello_state1 were not used when initializing DistilBertModel: ['classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
outputs=model(**batch,output_hidden_states=True,output_attentions=True)
all_hidden_states,all_attentions=outputs[-2:]

In [11]:
from transformers import DistilBertConfig,DistilBertTokenizer,DistilBertForSequenceClassification
config=DistilBertConfig(n_heads=8,dim=512,hidden_dim=4*512)
tokenizer=DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model=DistilBertForSequenceClassification(config)

In [12]:
from transformers import DistilBertTokenizer,DistilBertForSequenceClassification
model_name="distilbert-base-uncased"
model=DistilBertForSequenceClassification.from_pretrained(model_name,num_labels=10)
tokenizer=DistilBertTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifi

In [13]:
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained("bert-base-uncased")
sequence="A Titan RTX has 24GB of VRAM"
tokenized_sequence=tokenizer.tokenize(sequence)
print(tokenized_sequence)
inputs=tokenizer(sequence)
encoded_sequence=inputs["input_ids"]
print(encoded_sequence)
decoded_sequence=tokenizer.decode(encoded_sequence)
print(decoded_sequence)

['a', 'titan', 'rt', '##x', 'has', '24', '##gb', 'of', 'vr', '##am']
[101, 1037, 16537, 19387, 2595, 2038, 2484, 18259, 1997, 27830, 3286, 102]
[CLS] a titan rtx has 24gb of vram [SEP]


In [14]:
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained("bert-base-cased")
sequence_a="This is a short sequence."
sequence_b="This is a rather long sequence. It is at least longer than the sequence A."
encoded_sequence_a=tokenizer(sequence_a)["input_ids"]
encoded_sequence_b=tokenizer(sequence_b)["input_ids"]
print(len(encoded_sequence_a),len(encoded_sequence_b))
padded_sequence=tokenizer([sequence_a,sequence_b],padding=True)
print(padded_sequence["input_ids"])
print(padded_sequence["attention_mask"])

8 19
[[101, 1188, 1110, 170, 1603, 4954, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1188, 1110, 170, 1897, 1263, 4954, 119, 1135, 1110, 1120, 1655, 2039, 1190, 1103, 4954, 138, 119, 102]]
[[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]


In [15]:
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained("bert-base-cased")
sequence_a="HuggingFace is based in NYC"
sequence_b="Where is HuggingFace based?"
encoded_dict=tokenizer(sequence_a,sequence_b)
decoded=tokenizer.decode(encoded_dict["input_ids"])
print(decoded)
print(encoded_dict['token_type_ids'])

[CLS] HuggingFace is based in NYC [SEP] Where is HuggingFace based? [SEP]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [16]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
import torch

tokenizer=AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model=AutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")
classed=["not paraphrase","is paraphrase"]
sequence_0="The company HuggingFace is based in New York City"
sequence_1="Apples are especially bad for your health"
sequence_2="HuggingFace's headquarters are situated in Manhattan"
paraphrase=tokenizer(sequence_0,sequence_2,return_tensors="pt")
not_paraphrase=tokenizer(sequence_0,sequence_1,return_tensors="pt")
paraphrase_classification_logits=model(**paraphrase).logits
not_paraphrase_classification_logits=model(**not_paraphrase).logits
paraphrase_results=torch.softmax(paraphrase_classification_logits,dim=1).tolist()[0]
not_paraphrase_results=torch.softmax(not_paraphrase_classification_logits,dim=1).tolist()[0]
for i in range(len(classes)):
	print(f"{classes[i]}: {int(round(paraphrase_results[i] * 100))}%")

for i in range(len(classes)):
	print(f"{classes[i]}: {int(round(not_paraphrase_results[i] * 100))}%")

not paraphrase: 10%
is paraphrase: 90%
not paraphrase: 94%
is paraphrase: 6%


In [17]:
from transformers import pipeline

nlp=pipeline("question-answering")
context = r"""
Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a
question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune
a model on a SQuAD task, you may leverage the examples/question-answering/run_squad.py script.
"""

result = nlp(question="What is extractive question answering?", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")
result = nlp(question="What is a good example of a question answering dataset?", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")


No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


Answer: 'the task of extracting an answer from a text given a question', score: 0.6226, start: 34, end: 95
Answer: 'SQuAD dataset', score: 0.5053, start: 147, end: 160
